# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE
    # and then back ...
    supercell = ase_atoms(lattice, atoms, positions) * (repeat, 1, 1)
    lattice   = load_lattice(supercell)
    positions = load_positions(supercell)
    atoms = fill(Al, length(positions))

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(lattice, atoms, positions; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
ase_atoms(setup.model).write("al_supercell.png")

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
is_converged = DFTK.ScfConvergenceDensity(1e-4)  # Flag convergence based on density
self_consistent_field(aluminium_setup(1); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298650386561                   -0.85    4.0
  2   -8.300225066673       -2.80       -1.25    1.5
  3   -8.300448948380       -3.65       -1.87    4.5
  4   -8.300462251490       -4.88       -2.72    1.1
  5   -8.300464242455       -5.70       -3.14    3.9
  6   -8.300464432801       -6.72       -3.28   10.0
  7   -8.300464531617       -7.01       -3.41    2.4
  8   -8.300464588223       -7.25       -3.54    1.0
  9   -8.300464633124       -7.35       -3.77    1.1
 10   -8.300464639616       -8.19       -3.90    2.0
 11   -8.300464643577       -8.40       -4.31    1.2


In [4]:
self_consistent_field(aluminium_setup(2); is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.64202931594                   -0.70    5.4
  2   -16.67871086432       -1.44       -1.14    1.6
  3   -16.67921621827       -3.30       -1.87    3.2
  4   -16.67928176238       -4.18       -2.75    3.4
  5   -16.67928609937       -5.36       -3.11    5.8
  6   -16.67928621439       -6.94       -3.52    1.2
  7   -16.67928622199       -8.12       -4.02    1.4


In [5]:
self_consistent_field(aluminium_setup(4); is_converged, n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32375556870                   -0.56    5.9
  2   -33.33236581791       -2.06       -1.00    1.1
  3   -33.33542128948       -2.51       -1.69    3.4
  4   -33.33689192389       -2.83       -2.44    4.5
  5   -33.33693936424       -4.32       -2.93   10.4
  6   -33.33694386145       -5.35       -3.82    4.9
  7   -33.33694392128       -7.22       -4.22    7.9


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); is_converged, mixing=SimpleMixing());

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298529265080                   -0.85    4.5
  2   -8.300272826725       -2.76       -1.59    1.0
  3   -8.300449182045       -3.75       -2.54    6.5
  4   -8.300368997000   +   -4.10       -2.27   13.4
  5   -8.300464576362       -4.02       -3.78    3.0
  6   -8.300464607484       -7.51       -4.02    3.9


In [7]:
self_consistent_field(aluminium_setup(4); is_converged, mixing=SimpleMixing(), n_bands=30);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32593468919                   -0.56    6.0
  2   -33.23476519910   +   -1.04       -1.23    1.0
  3   +37.47412638996   +    1.85       -0.14   11.0
  4   -33.30717270224        1.85       -1.65    8.2
  5   -33.26167647675   +   -1.34       -1.36    4.5
  6   -32.98289034805   +   -0.55       -1.29    5.4
  7   -33.20182048656       -0.66       -1.49    4.5
  8   -33.33594847030       -0.87       -2.18    3.6
  9   -33.33303191780   +   -2.54       -2.02    2.8
 10   -33.33672074872       -2.43       -2.59    2.9
 11   -33.33691696656       -3.71       -3.01    2.1
 12   -33.33692482419       -5.10       -3.16    3.0
 13   -33.33693453906       -5.01       -3.37    1.8
 14   -33.33694369415       -5.04       -4.01    2.9


For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.